<a href="https://colab.research.google.com/github/wfos3241/TextMining/blob/main/ex03_%ED%95%9C%EA%B8%80%ED%98%90%EC%98%A4%ED%91%9C%ED%98%84%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 학습목표
- 빈도분석
- 띄어쓰기 교정하기
- Kiwi를 이용한 불용어 처리
- 교차검증
- 하이퍼파라미터 튜닝 (GridSearch)
- GBM 계열 모델 : Grandient Boost Machine(GBM), Xgboost, LightGBM

# 데이터셋
- 스마일게이트의 한국어 혐오표현 데이터셋
- 여성/가족, 남성, 성소수자, 인종/국적, 연령, 지역, 종교, 혐오, 악플, 욕설, clean, 개인지칭, 기타 혐오표현으로 구성
- 15005개의 훈련세트와 3737개의 테스트세트로 구성

In [19]:
from google.colab import drive

In [20]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
%cd /content/drive/MyDrive/Colab Notebooks/텍스트마이닝

/content/drive/MyDrive/Colab Notebooks/텍스트마이닝


In [22]:
import pandas as pd

train = pd.read_csv("./data/unsmile_train_v1.0.tsv", delimiter = "\t")
test = pd.read_csv("./data/unsmile_valid_v1.0.tsv", delimiter = "\t")

In [23]:
display(train.head())
display(test.head())

,문장,여성/가족,남성,성소수자,인종/국적,연령,지역,종교,기타 혐오,악플/욕설,clean,개인지칭
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,0,0,0,0,0,0,0,0,0,1,0
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,0,0,0,0,0,0,1,0,0,0,0
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,0,0,0,0,0,0,0,0,0,1,0
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,0,0,0,0,0,0,0,0,0,1,0
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,1,0,0,0,0,0,0,0,0,0,0


,문장,여성/가족,남성,성소수자,인종/국적,연령,지역,종교,기타 혐오,악플/욕설,clean,개인지칭
0,ㅇㄱㄹㅇ 진짜 죽어도 상관없다는 마인드로 싸웠더니 지금 서열 상타취노 식칼들고 니가...,0,1,0,0,0,0,0,0,0,0,0
1,여자들은 취미가 애낳는건가.. 취미를 좀 가져라,1,0,0,0,0,0,0,0,0,0,0
2,개슬람녀 다 필요없고 니 엄마만 있으면 된다,0,0,0,1,0,0,1,0,0,0,0
3,조팔ㅋㅋ 남한 길거리 돌아다니면 한국남자때문에 눈재기하는데 그걸 내 폰에 굳이 담아...,0,1,0,0,0,0,0,0,0,0,0
4,바지 내리다 한남들 와꾸 보고 올려뿟노,0,1,0,0,0,0,0,0,0,0,0


In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15005 entries, 0 to 15004
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   문장      15005 non-null  object
 1   여성/가족   15005 non-null  int64 
 2   남성      15005 non-null  int64 
 3   성소수자    15005 non-null  int64 
 4   인종/국적   15005 non-null  int64 
 5   연령      15005 non-null  int64 
 6   지역      15005 non-null  int64 
 7   종교      15005 non-null  int64 
 8   기타 혐오   15005 non-null  int64 
 9   악플/욕설   15005 non-null  int64 
 10  clean   15005 non-null  int64 
 11  개인지칭    15005 non-null  int64 
dtypes: int64(11), object(1)
memory usage: 1.4+ MB


In [25]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3737 entries, 0 to 3736
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   문장      3737 non-null   object
 1   여성/가족   3737 non-null   int64 
 2   남성      3737 non-null   int64 
 3   성소수자    3737 non-null   int64 
 4   인종/국적   3737 non-null   int64 
 5   연령      3737 non-null   int64 
 6   지역      3737 non-null   int64 
 7   종교      3737 non-null   int64 
 8   기타 혐오   3737 non-null   int64 
 9   악플/욕설   3737 non-null   int64 
 10  clean   3737 non-null   int64 
 11  개인지칭    3737 non-null   int64 
dtypes: int64(11), object(1)
memory usage: 350.5+ KB


- 혐오표현별 자주 사용되는 단어의 빈도수를 조사

In [26]:
test.columns

Index(['문장', '여성/가족', '남성', '성소수자', '인종/국적', '연령', '지역', '종교', '기타 혐오',
       '악플/욕설', 'clean', '개인지칭'],
      dtype='object')

In [27]:
# 지역관련 혐오표현에 사용되는 단어는 어떤 것들이 있는지

text_train = train[train["성소수자"] == 1]["문장"]
text_train

,문장
10,가짜 남자는 어떻게 생겼냐?
38,돈고는 질병이다 이기야
42,동성애는 저주야!
61,로봉순 젠신병자니까 맘껏패라 사람아님
67,똥고에 들어갔던 꼬치 먹고싶은사람이 없어서 그렇지않겠노
...,...
14928,동성애자 표가 얼마나 된다고 무기타령 하고 있네. 겜대리녀가 정의를 논하는 것도 우습고.
14935,똥꼬새키들은 죽어도 돼
14940,안타까운 일이지만 이번 기회에 사회와 국가에서 바른 성정체성을 심어주어야 힙니다. ...
14954,ㅋㅋㅋㅋㅋ 트젠이 올렸을확률 69.696969696969%놐ㅋㅋㅋ


In [28]:
# 각 문장들을 토큰화 (빈 공백)

area_tokens = [t for doc in text_train for t in doc.split(" ")]
area_tokens

['가짜',
 '남자는',
 '어떻게',
 '생겼냐?',
 '돈고는',
 '질병이다',
 '이기야',
 '동성애는',
 '저주야!',
 '로봉순',
 '젠신병자니까',
 '맘껏패라',
 '사람아님',
 '똥고에',
 '들어갔던',
 '꼬치',
 '먹고싶은사람이',
 '없어서',
 '그렇지않겠노',
 '이새끼',
 '똥꼬충인거같은데',
 '똥꼬충새끼들이',
 '양성애자들',
 '존나',
 '고깝게',
 '보더라글제목부터가',
 '성소수자',
 '라고',
 '대우해주는게',
 '이새끼',
 '똥꼬충으로',
 '추정된다',
 '아니',
 '시벌',
 '터키',
 '그나마',
 '보수적',
 '마인드',
 '때문에',
 '좋아했는데',
 '동성애자들이',
 '저정도로',
 '날뛰는거면',
 '답없네',
 '아직까진',
 '불필요하다',
 '여자',
 '남자속에서',
 '숨어살면서',
 '또',
 '소수자를',
 '만들고',
 '지네들이',
 '하는게뭔데....불편해',
 '그냥',
 '숨어살어',
 '똥꼬충을',
 '권유하는',
 '시대까지오노',
 '난',
 'homosexual을',
 '지지하는',
 'heterosexual인데',
 '솔직히',
 '저들의',
 'way',
 'to',
 'express는',
 '좀',
 'seriously',
 '문제가',
 '있다고',
 '생각한다',
 '제발',
 '성에',
 '관해서만은',
 '관대하지',
 '맙시다!',
 '인류의',
 '미래와',
 '연관되어',
 '있다고',
 '봅니다.',
 '성소수자',
 '차별금지법',
 '절대반대!!!',
 '성다수자가',
 '역차별',
 '당하는법!!!',
 '남녀가',
 '헐벗고',
 '섹스하자고',
 '춤추고',
 '퍼레이드',
 '하고',
 '콘돔',
 '나눠주면',
 '바로',
 '잡혀갈텐데동성연애자들이',
 '춤추고',
 '항문섹스하자고',
 '콘돔',
 '나눠주는',
 '것은',
 '왜',
 '가만둘까?',
 '에이즈의',
 '주범이',
 '똥꼬충인거',
 '이미',

In [29]:
# 지역과 관련되서 자주 쓰이는 단어의 빈도수를 확인

from collections import Counter

area_counter = Counter(area_tokens)
area_counter.most_common(20)

# 상위 20개만 출력

[('다', 78),
 ('동성애', 78),
 ('왜', 70),
 ('똥꼬충', 59),
 ('동성애는', 54),
 ('그냥', 39),
 ('진짜', 38),
 ('에이즈', 36),
 ('더', 30),
 ('동성애를', 29),
 ('성소수자', 27),
 ('아니라', 27),
 ('좀', 26),
 ('이', 25),
 ('', 24),
 ('존나', 23),
 ('절대', 23),
 ('똥꼬충들', 23),
 ('ㅋㅋ', 22),
 ('정말', 22)]

- 정제
  - 정규식으로 한글과 공백만 추출
  - 1 글자로만 구성된 토큰은 제외

In [30]:
import re

# 한글과 빈공백이 아닌 것

p = re.compile("[^가-힣\s]")

# 결과 저장

train_clean = []

for doc in train["문장"] :

  # 중간 결과 저장

  temp = []

  # 문장을 토큰화하고 토큰을 하나씩 불러옴

  for token in doc.split(" ") :

    # 한글자인 토큰 제외

    if len(token) < 2 :
      continue

    # 정규식 표현에 해당하는 토큰 제외

    if p.search(token) :
      continue

    temp.append(token)

  # 토큰들을 빈공백으로 구분하여 하나의 문장으로 리스트에 저장

  train_clean.append(" ".join(temp))

In [31]:
# 한글과 빈공백이 아닌 것

p = re.compile("[^가-힣\s]")

# 결과 저장

test_clean = []

for doc in test["문장"] :

  # 중간 결과 저장

  temp = []

  # 문장을 토큰화하고 토큰을 하나씩 불러옴

  for token in doc.split(" ") :

    # 한글자인 토큰 제외

    if len(token) < 2 :
      continue

    # 정규식 표현에 해당하는 토큰 제외

    if p.search(token) :
      continue

    temp.append(token)

  # 토큰들을 빈공백으로 구분하여 하나의 문장으로 리스트에 저장

  test_clean.append(" ".join(temp))

- 띄어쓰기 교정
  - kiwi 라이브러리 사용

In [32]:
!pip install -q kiwipiepy konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.5/35.5 MB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 19.9 MB/s eta 0:00:00


In [33]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

# space() : 띄어쓰기를 교정하는 함수

kiwi.space("나는학교에갑니다안녕하세요반갑습니다")

'나는 학교에 갑니다 안녕하세요 반갑습니다'

- 훈련/검증 데이터의 띄어쓰기 교정

In [45]:
train_clean2 = list(kiwi.space(train_clean))
test_clean2 = list(kiwi.space(test_clean))

- 불용어 처리

In [36]:
from kiwipiepy.utils import Stopwords

stopwords = Stopwords()

kiwi.tokenize("분석 결과에서 불용어만 제외하고 출력할 수도 있다", stopwords = stopwords)

stopwords.add(("있", "VA"))

kiwi.tokenize("분석 결과에서 불용어만 제외하고 출력할 수도 있다", stopwords = stopwords)

[Token(form='분석', tag='NNG', start=0, len=2),
 Token(form='결과', tag='NNG', start=3, len=2),
 Token(form='불', tag='NNG', start=8, len=1),
 Token(form='용어', tag='NNG', start=9, len=2),
 Token(form='제외', tag='NNG', start=13, len=2),
 Token(form='출력', tag='NNG', start=18, len=2)]

- 전처리된 데이터를 형태소 분리, 불용어 처리, 토큰 연결

In [46]:
def clean_and_tokenize(text, tokenizer) :
  #불용어 제거

  tokens = kiwi.tokenize(text, stopwords = stopwords)

  # 토큰 이어 붙이기
  # token.form : kiwi 토큰 객체의 form 속성 값을 가져온다

  clean_text = " ".join([token.form for token in tokens])

  # 형태소 분리

  clean_text2 = tokenizer.morphs(clean_text)

  # CountVectorizer, TfidVectorizer의 입력이 문장형태

  return " ".join(clean_text2)

In [44]:
train_clean2


list[<map object at 0x7b901a252620>]

In [47]:
from konlpy.tag import Okt

okt = Okt()

train_clean3 = []

for doc in train_clean2 :
  train_clean3.append(clean_and_tokenize(doc, okt))

test_clean3 = []

for doc in test_clean2 :
  test_clean3.append(clean_and_tokenize(doc, okt))

- 인코딩

In [48]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cv = CountVectorizer(
    ngram_range = (1, 2), # 토큰이 1개와 2개인 것
                          # 토큰 : [나는, 학교에, 갑니다]
                          # ngram이 (1, 2)인 경우 : 나는, 학교에, 갑니다, 나는 학교에, 나는 갑니다, 학교에 갑니다
    max_df = 0.9, # 70% 이상의 문서에서 등장하는 단어는 사용하지 않음
    min_df = 3 # 빈도수가 10이하인 토큰은 사용하지 않음
    )
tv = TfidfVectorizer(
    ngram_range = (1, 2),
    max_df = 0.9,
    min_df = 3
    )



In [49]:
cv.fit(train_clean3)
tv.fit(train_clean3)

TfidfVectorizer(max_df=0.9, min_df=3, ngram_range=(1, 2))

In [50]:
X_train_cv = cv.transform(train_clean3)
X_train_tv = tv.transform(train_clean3)
X_test_cv = cv.transform(test_clean3)
X_test_tv = tv.transform(test_clean3)

- 라벨 데이터 처리
  - 혐오표현 : 0
  - 악플&욕설 : 1
  - clean : 2
  - 개인지칭 : 나중에 추가된 항목이라 제외

In [52]:
# 여성/가족 컬럼부터 기타 혐오 컬럼까지 혐오표현 컬럼으로 통합
# 각 컬럼의 값을 합해서 혐오표현 컬럼에 저장

train["혐오표현"] = train.loc[:, "여성/가족":"기타 혐오"].sum(axis = 1)
test["혐오표현"] = test.loc[:, "여성/가족":"기타 혐오"].sum(axis = 1)

In [53]:
train.head()

,문장,여성/가족,남성,성소수자,인종/국적,연령,지역,종교,기타 혐오,악플/욕설,clean,개인지칭,혐오표현
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,0,0,0,0,0,0,0,0,0,1,0,0
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,0,0,0,0,0,0,1,0,0,0,0,1
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,0,0,0,0,0,0,0,0,0,1,0,0
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,0,0,0,0,0,0,0,0,0,1,0,0
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,1,0,0,0,0,0,0,0,0,0,0,1


In [55]:
# 통합된 컬럼 삭제

new_train = train[["문장", "혐오표현", "악플/욕설", "clean"]]
new_test = test[["문장", "혐오표현", "악플/욕설", "clean"]]

- 3개의 라벨 컬럼 (혐오표현, 악플/욕설, clean)을 이용해서 라벨 데이터 생성

In [57]:
def label(row) :
  if row["혐오표현"] > 0 :
    return 0
  elif row["악플/욕설"] > 0 :
    return 1
  else :
    return 2

In [58]:
# 위에서 정의한 label() 함수에 전체 데이터를 전송해서 라벨 데이터를 생성

y_train = new_train.apply(label, axis = 1)
y_test = new_test.apply(label, axis = 1)

In [59]:
# 라벨 데이터의 편향 확인

print(y_train.value_counts())
print(y_test.value_counts())

0    8121
2    3741
1    3143
Name: count, dtype: int64
0    2016
2     935
1     786
Name: count, dtype: int64


- 학습
  - LogisticRegression()
    - C
    - max_iter
  - RandomForest
    - random_state
    - n_estimators

In [88]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

lr = LogisticRegression(C = 10, max_iter = 1000)
rf = RandomForestClassifier(random_state = 40, n_estimators = 100)
gbm = GradientBoostingClassifier(random_state = 42)

lr.fit(X_train_cv, y_train)
lr.fit(X_train_tv, y_train)
rf.fit(X_train_cv, y_train)
rf.fit(X_train_tv, y_train)
gbm.fit(X_train_cv, y_train)
gbm.fit(X_train_tv, y_train)

GradientBoostingClassifier(random_state=42)

In [89]:
print(lr.score(X_test_cv, y_test))
print(lr.score(X_test_tv, y_test))
print(rf.score(X_test_cv, y_test))
print(rf.score(X_test_tv, y_test))
print(gbm.score(X_test_cv, y_test))
print(gbm.score(X_test_tv, y_test))

0.6408884131656409
0.6317902060476318
0.6047631790206047
0.6392828472036393
0.5474979930425475
0.5579341717955579


- 교차검증
  - 과적합이 나오는 정확도를 일반화 점수화 하는 작업

In [75]:
from sklearn.model_selection import cross_val_score

# 교차검증

score_lr_cv = cross_val_score(lr, X_train_cv, y_train, cv = 5)
score_lr_tv = cross_val_score(lr, X_train_tv, y_train, cv = 5)

In [77]:
print("CounterVectorizer() + LogisticRegression()")
print(score_lr_cv)
print(score_lr_cv.mean())
print("TfidfVectorizer() + LogisticRegression()")
print(score_lr_tv)
print(score_lr_tv.mean())

CounterVectorizer + LogisticRegression
[0.61146285 0.61346218 0.61179607 0.59946684 0.59846718]
0.6069310229923358
TfidfVectorizer + LogisticRegression
[0.62379207 0.62179274 0.61412862 0.60313229 0.60546485]
0.6136621126291236


In [83]:
score_rf_cv = cross_val_score(rf, X_train_cv, y_train, cv = 5)
score_rf_tv = cross_val_score(rf, X_train_tv, y_train, cv = 5)

In [84]:
print("CounterVectorizer() + RandomForest()")
print(score_rf_cv)
print(score_rf_cv.mean())
print("TfidfVectorizer() + RandomForest()")
print(score_rf_tv)
print(score_rf_tv.mean())

CounterVectorizer() + RandomForest()
[0.61579474 0.62045985 0.62312562 0.61112962 0.62112629]
0.6183272242585804
TfidfVectorizer() + RandomForest()
[0.63512163 0.62845718 0.63212263 0.62679107 0.63745418]
0.6319893368877041


- GBM 모델 적용하기
  - 앙상블 모델
    - Bagging 방식
      - 데이터를 일부만 샘플링 -> 여러 개의 결과값
      - 각 학습 결과들을 투표하여 결과값이 많은 것을 선택
      - 대표 모델 : RandomForest
    - Boosting 방식
      - 전체 데이터를 이용하여 학습 -> 하나의 결과 -> 전체 데이터에서 일부데이터의 값을 변경 -> 하나의 결과 -> 반복
      - 가중 평균값으로 결정
      - 대표 모델 : GBM

In [85]:
from sklearn.ensemble import GradientBoostingClassifier

gbm = GradientBoostingClassifier(random_state = 42)

In [86]:
score_gbm_cv = cross_val_score(gbm, X_train_cv, y_train, cv = 5)
score_gbm_tv = cross_val_score(gbm, X_train_tv, y_train, cv = 5)

In [87]:
print("CounterVectorizer() + GradientBoostingClassifier()")
print(score_gbm_cv)
print(score_gbm_cv.mean())
print("TfidfVectorizer() + GradientBoostingClassifier()")
print(score_gbm_tv)
print(score_gbm_tv.mean())

CounterVectorizer() + GradientBoostingClassifier()
[0.55214928 0.54715095 0.54615128 0.5434855  0.54215262]
0.5462179273575474
TfidfVectorizer() + GradientBoostingClassifier()
[0.55414862 0.56614462 0.56047984 0.55581473 0.55114962]
0.5575474841719427
